In [1]:
import pandas as pd

In [2]:
def category_to_labelEncoding(dataframeOriginal,columna):
    dataframe=dataframeOriginal.copy()
    dataframe[columna]=dataframe[columna].astype('category')
    all_in_a_column= columna + '_' + 'cat_codes'
    dataframe[all_in_a_column]= dataframe[columna].cat.codes #puede dar orden de importancia que se puede mal interpretar
    dataframe=pd.get_dummies(dataframe, columns=[columna])
    return dataframe

# PROCESO EL TRAIN


In [3]:
df_FE_1_train=pd.read_csv('FE_1_train.csv')
df_FE_1_test=pd.read_csv('FE_1_test.csv')

qContinents=8
qCountries=10
qCities=10


topContinent = df_FE_1_train.continent.value_counts().head(qContinents)
df_FE_1_train.loc[(~df_FE_1_train.continent.isin(topContinent.index)), 'continent'] = 'Other'
df_FE_1_train.loc[(df_FE_1_train.continent.isin(['__UNKNOWN'])), 'continent'] = 'Other'
lista_continents=df_FE_1_train.continent.value_counts().index

topCountries = df_FE_1_train.country.value_counts().head(qCountries)
df_FE_1_train.loc[(~df_FE_1_train.country.isin(topCountries.index)), 'country'] = 'Other'
df_FE_1_train.loc[(df_FE_1_train.country.isin(['__UNKNOWN'])), 'country'] = 'Other'
lista_countries=df_FE_1_train.country.value_counts().index

topCities = df_FE_1_train.city.value_counts().head(qCities)
df_FE_1_train.loc[(~df_FE_1_train.city.isin(topCities.index)), 'city'] = 'Other'
df_FE_1_train.loc[(df_FE_1_train.city.isin(['__UNKNOWN'])), 'city'] = 'Other'
lista_cities=df_FE_1_train.city.value_counts().index

df_FE_1_train=category_to_labelEncoding(df_FE_1_train,'continent')
df_FE_1_train=category_to_labelEncoding(df_FE_1_train,'country')
df_FE_1_train=category_to_labelEncoding(df_FE_1_train,'city')
df_train=df_FE_1_train.drop(columns=['Unnamed: 0','keyword','location','text','complete_text'])*1

df_train_T5=pd.read_csv('T5_fe_1.csv')
df_train_T5.columns=['id','prediccion_T5']
df_train=df_train.merge(df_train_T5, on='id')

# PROCESO EL TEST

In [4]:
df_FE_1_test.loc[(~df_FE_1_test.continent.isin(lista_continents)), 'continent'] = 'Other'
df_FE_1_test.loc[(df_FE_1_test.continent.isin(['__UNKNOWN'])), 'continent'] = 'Other'

df_FE_1_test.loc[(~df_FE_1_test.country.isin(lista_countries)), 'country'] = 'Other'
df_FE_1_test.loc[(df_FE_1_test.country.isin(['__UNKNOWN'])), 'country'] = 'Other'

df_FE_1_test.loc[(~df_FE_1_test.city.isin(lista_cities)), 'city'] = 'Other'
df_FE_1_test.loc[(df_FE_1_test.city.isin(['__UNKNOWN'])), 'city'] = 'Other'   

df_FE_1_test=category_to_labelEncoding(df_FE_1_test,'continent')
df_FE_1_test=category_to_labelEncoding(df_FE_1_test,'country')
df_FE_1_test=category_to_labelEncoding(df_FE_1_test,'city')
df_test=df_FE_1_test.drop(columns=['Unnamed: 0','keyword','location','text','complete_text'])*1

df_test_T5=pd.read_csv('T5_submit_1.csv')
df_test_T5.columns=['id','prediccion_T5']
df_test=df_test.merge(df_test_T5, on='id')


# GUARDO LOS CSV PROCESADOS

In [5]:
df_train.to_csv('FE_v2_train.csv',index=False)
df_test.to_csv('FE_v2_test.csv',index=False)

In [6]:
df1=pd.read_csv('FE_v2_train.csv')
df1.head()

,id,target,hasNum,hasLink,hasUser,hasHashtag,exclamacion,interrogacion,mayusculas,longitud,...,city_Chicago,city_Florida,city_London,city_Los Angeles,city_New York,city_Of,city_Other,city_Texas,city_Washington,prediccion_T5
0,1,1,0,0,0,1,0,0,10,71,...,0,0,0,0,0,0,1,0,0,1
1,4,1,0,0,0,0,0,0,5,40,...,0,0,0,0,0,0,1,0,0,1
2,5,1,0,0,0,0,0,0,2,135,...,0,0,0,0,0,0,1,0,0,1
3,6,1,1,0,0,1,0,0,1,67,...,0,0,0,0,0,0,1,0,0,1
4,7,1,0,0,0,1,0,0,3,90,...,0,0,0,0,0,0,1,0,0,1


In [7]:
df2=pd.read_csv('FE_v2_test.csv')
df2.head()

,id,hasNum,hasLink,hasUser,hasHashtag,exclamacion,interrogacion,mayusculas,longitud,negativity,...,city_Chicago,city_Florida,city_London,city_Los Angeles,city_New York,city_Of,city_Other,city_Texas,city_Washington,prediccion_T5
0,0,0,0,0,0,0,0,1,36,0,...,0,0,0,0,0,0,1,0,0,1
1,2,0,0,0,1,0,0,1,66,0,...,0,0,0,0,0,0,1,0,0,1
2,3,0,0,0,0,0,0,1,98,1,...,0,0,0,0,0,0,1,0,0,1
3,9,0,0,0,1,0,0,2,42,0,...,0,0,0,0,0,0,1,0,0,1
4,11,1,0,0,0,0,0,4,47,0,...,0,0,0,0,0,0,1,0,0,1


In [8]:
for x in df1.columns:
    if x in df2.columns:
        None
    else:
        print(x)

target
